### N Gram In N Gram Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [18]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [ ]:
def word_in_wordgroup(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_twogram = pd.read_excel(f"{lang_folder.capitalize()}_Shared_Twogram.xlsx")
df_twogram = df_twogram.loc[:,["twogram","frequency"]]
df_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_twogram

In [20]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    df_twogram = df_twogram.iloc[word_start:word_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    df_twogram = df_twogram.iloc[word_start:word_end,]

df_twogram

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
195,ayağa kalk,11389
196,tanrıya şükür,11306
197,evet var,11138
198,yarın görüşürüz,11136


In [10]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,ne demek istiyorsun,55905
1,bu da ne,50098
2,hayır hayır hayır,41386
3,sen iyi misin,32965
4,çok teşekkür ederim,27663
...,...,...
1039403,kapıyı kapat aaron,5
1039404,kardeşime bir hediye,5
1039405,sonra da rachampsı,5
1039406,usta lee neredesin,5


In [14]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,her şey yolunda mı,23401
1,bir sorun mu var,14906
2,burada ne işin var,14392
3,ben de seni seviyorum,12348
4,her şey yoluna girecek,11682
...,...,...
1040253,yine de rahatladığımı söyleyemem,5
1040254,sen gelip bana saldırdın,5
1040255,i̇yi arkadaşlık bundan ibarettir,5
1040256,ben dr elizabeth weir,5


In [19]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4
